In [5]:
from pystac_client import Client
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, box
import logging

# Set up logging to track API calls
logging.basicConfig()
logger = logging.getLogger('pystac_client')
logger.setLevel(logging.INFO)

# Read the shapefile to determine the bounding box
import geopandas as gpd
from pyproj import CRS, Transformer

# Define the custom Mars projection (replace with your Mars projection string)
mars_proj = CRS.from_proj4(
    "+proj=eqc +a=3396190 +b=3396190 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +units=m +no_defs"
)
# Define the target Mars lon/lat coordinate system
mars_lonlat = CRS.from_proj4(
    "+proj=longlat +a=3396190 +b=3396190 +no_defs"
)
# Create a transformer
transformer = Transformer.from_crs(mars_proj, mars_lonlat, always_xy=True)
# Load the shapefile
shp_file = "roi_test.shp"
gdf = gpd.read_file(shp_file)
# Reproject the GeoDataFrame
gdf.to_crs(mars_lonlat.to_string(), inplace=True)
bounding_boxes = gdf.geometry.bounds.values
print("Transformed Bounding Box (Target Projection):", bounding_boxes)
bounding_box = bounding_boxes[0]

Transformed Bounding Box (Target Projection): [[76.97670509 17.72565618 77.16420794 17.9416318 ]]


In [13]:
# Set up the STAC catalog client
catalog_url = "https://stac.astrogeology.usgs.gov/api/"
catalog = Client.open(catalog_url)

# List of databases to query
databases = ['mro_ctx_controlled_usgs_dtms']
results = []

# Perform the query for each database
for database in databases:
    result = catalog.search(collections=[database], bbox=bounding_box)
    results.append(result)

# Get all items as a dictionary
items = []
for r in results:
    items += r.item_collection_as_dict()['features']
for i in items:
    i['geometry'] = shape(i['geometry'])
items_gdf = gpd.GeoDataFrame(pd.json_normalize(items))
# Verify the retrieved items
print(f"Number of items retrieved: {len(items_gdf)}")

Number of items retrieved: 33


In [14]:
# Write URLs to a text file
output_file = "ctx_dtm_to_download.txt"
with open(output_file, 'w') as f:
    for index in range(len(items_gdf)):
        old_url = items_gdf['assets.image.href'].iloc[index]
        updated_url = f"s3_noauth:/{old_url.split('/')[2].split('.')[0]}/{'/'.join(old_url.split('/')[3:])}\n"
        f.write(updated_url)

print(f"URLs have been written to '{output_file}'.")

URLs have been written to 'ctx_dtm_to_download.txt'.
